### moduls install

In [ ]:
!pip install tensorflow==2.9.0
!pip install "numpy<2.0"
!pip install matplotlib seaborn scikit-learn opencv-python-headless

### import moduls

In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

### ⭐️ dir setting

In [ ]:
model_dir = 'best_model'
structure_path = os.path.join(model_dir, 'model_structure.json')
weights_path = os.path.join(model_dir, 'model_weights.h5')
config_path = os.path.join(model_dir, 'config.json')
class_info_path = os.path.join(model_dir, 'class_info.json')

### model loading

In [ ]:

# 모델 + 클래스 정보 불러오기 (추론 전용)
def load_inference_model(model_dir):
    # 모델 구조 + 가중치
    with open(os.path.join(model_dir, 'model_structure.json')) as f:
        model = model_from_json(f.read())
    model.load_weights(os.path.join(model_dir, 'model_weights.h5'))

    # class_info.json 로드
    with open(os.path.join(model_dir, 'class_info.json')) as f:
        class_info = json.load(f)
    
    # 클래스 인덱스 → 라벨 딕셔너리 생성
    idx_to_class = {entry['index']: entry['class'] for entry in class_info}

    # 입력 이미지 크기
    img_height = class_info[0]['height']
    img_width = class_info[0]['width']

    return model, idx_to_class, (img_height, img_width)


### ⭐️ input data

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
img_path = 'test_images/sample.jpg'  # ⭐️ 여기에 새 이미지 경로
img = load_img(img_path, target_size=(224, 224))  # ⭐️ 모델 입력 사이즈에 맞게 수정!
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:

# 전처리 함수
def preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # 정규화
    img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가
    return img_array

### run

In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
# path setting
structure_path = 'best_model/model_structure.json'
weights_path = 'best_model/model_weights.h5'
config_path = 'best_model/config.json'
class_names_path = 'best_model/class_names.json'

# model, class loading
model, config = load_model_with_config(structure_path, weights_path, config_path)

with open(class_names_path, 'r') as f:
    class_names = json.load(f)


In [ ]:
# 후처리 함수
def decode_prediction(preds, idx_to_class):
    pred_index = np.argmax(preds, axis=1)[0]
    pred_class = idx_to_class[pred_index]
    confidence = float(preds[0][pred_index])
    return pred_class, confidence

# 예측 함수
def predict_image(model, img_path, img_size, idx_to_class):
    img = preprocess_image(img_path, target_size=img_size)
    preds = model.predict(img)
    pred_class, confidence = decode_prediction(preds, idx_to_class)
    return pred_class, confidence

#### ⭐️ model activate

In [ ]:
# running model
img_path = 'your_image.jpg'  # ⭐️ 예측할 이미지 경로
pred_class, confidence = predict_image(img_path, model, class_names)

print(f"📌 예측 클래스: {pred_class} ({confidence * 100:.2f}%)")